# Example Drift Detection with Evidently

This notebook provides an example of using Evidently to assess drift as discussed in [https://bradleyboehmke.github.io/uc-bana-7075/09-modelops-monitoring.html#examples-of-implementing-evidently](https://bradleyboehmke.github.io/uc-bana-7075/09-modelops-monitoring.html#examples-of-implementing-evidently).

In [1]:
from apple_data import generate_apple_sales_data_with_promo_adjustment

# Generate baseline dataset (no drift)
baseline_df = generate_apple_sales_data_with_promo_adjustment(
    n_rows=5000
)

# Generate drifted dataset with controlled drift factors
feature_drift_config = {
    "average_temperature": 1.2,  # 20% increase
    "rainfall": 0.9,             # 10% decrease
    "price_per_kg": 1.09,        # 9% increase
    "promo": 1.5                 # 50% more promotions
}

concept_drift_config = {
    "price_sensitivity": 0.8,  # 20% less sensitive to price changes
    "promo_effect": 0.7,       # Promotions become 30% less effective
    "weekend_effect": 1.1,     # Weekend demand slightly increases
    "feature_importance": True
}

drifted_df = generate_apple_sales_data_with_promo_adjustment(
    n_rows=5000,
    feature_drift_factors=feature_drift_config,
    concept_drift_factors=concept_drift_config
)

In [2]:
# save drifted data to demo later
drifted_df.to_csv("drifted_batch_data.csv", index=False)

## Feature Drift

In [3]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Define the drift detection report
data_drift_report = Report(metrics=[DataDriftPreset(drift_share=0.3)])

# Run comparison
data_drift_report.run(reference_data=baseline_df, current_data=drifted_df)

# Save the report
data_drift_report.save_html("data_drift_report.html")

## Target/Concept Drift

In [4]:
from evidently.metric_preset import TargetDriftPreset
from evidently import ColumnMapping


# Define the target variable and date variable
column_mapping = ColumnMapping()
column_mapping.target = 'demand'
column_mapping.datetime = 'date'

# Create an Evidently AI report for concept drift detection
concept_drift_report = Report(metrics=[TargetDriftPreset()])

# Run the report specifying the target variable
concept_drift_report.run(
    reference_data=baseline_df,
    current_data=drifted_df,
    column_mapping=column_mapping  # Explicitly map the target variable
)

# Save and view the report
concept_drift_report.save_html("concept_drift_report.html")


## Model Performance Drift

In [5]:
from evidently.metric_preset import RegressionPreset
import mlflow


# Load current model
mlflow.set_experiment("Forecasting Apple Demand")
MODEL_URI = "models:/apple_demand@champion"
model = mlflow.pyfunc.load_model(MODEL_URI)

# Score baseline & drift data with model
baseline_df['predictions'] = model.predict(drifted_df)
drifted_df['predictions'] = model.predict(drifted_df)

# Add the prediction variable to our column mapping to compare to actuals
column_mapping.prediction = 'predictions'

# Create a model performance monitoring report
performance_report = Report(metrics=[RegressionPreset()])
performance_report.run(
    reference_data=baseline_df,
    current_data=drifted_df,
    column_mapping=column_mapping  # Explicitly map the target variable
)

# Save report
performance_report.save_html("performance_report.html")


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

